# Introduction to the workflow structure

This notebook gives a brief overview of the workflow structure and introduce some useful DataJoint tools to facilitate the exploration.
+ DataJoint needs to be pre-configured before running this notebook, if you haven't set up the configuration, refer to notebook [00-Set-up-configuration](00-Set-up-configuration.ipynb).
+ If you are familar with DataJoint and the workflow structure, proceed to the next notebook [02-process-ephys-workflow](02-process-ephys-workflow.ipynb) directly to run the workflow.
+ For a more thorough introduction of DataJoint functionings, please visit our [datajoint tutorial site](https://playground.datajoint.io) or [tutorials for BrainCogs U19 team](../tutorials/202103)

To load the local configuration, we will change the directory to the package root.

In [ ]:
import os
os.chdir('../..')

## Ephys related schemas and tables

The current workflow is composed of multiple database schemas, each of them corresponds to a module within the database.

Some of the schemas and tables are from `u19_pipeline`, and the ephys related schemas are from the [DataJoint Array Ephys Element](https://github.com/datajoint/element-array-ephys) installed in `u19_pipeline`.

In [ ]:
import datajoint as dj
from u19_pipeline import subject, acquisition, ephys
from u19_pipeline.ephys import probe_element, ephys_element

+ Each module contains a schema object that enables interaction with the schema in the database.
+ The module `ephys` is the native 

In [ ]:
probe_element.schema

In [ ]:
ephys_element.schema

+ The table classes in the module corresponds to a table in the schema in the database.

In [ ]:
# preview table columns and contents in a table
ephys_element.EphysRecording()

+ By importing the modules for the first time, the schemas and tables will be created inside the database.
+ Once created, importing modules will not create schemas and tables again, but the existing schemas/tables can be accessed and manipulated by the modules.

## DataJoint tools to explore schemas and tables

+ `dj.list_schemas()`: list all schemas a user has access to in the current database

In [ ]:
dj.list_schemas()

+ `dj.Diagram()`: plot tables and dependencies. 

In [ ]:
# plot diagram for all tables in a schema
dj.Diagram(ephys_element)

**Table tiers**: 

Manual table: green box, manually inserted table, expect new entries daily, e.g. Subject, ProbeInsertion.  
Lookup table: gray box, pre inserted table, commonly used for general facts or parameters. e.g. Strain, ClusteringMethod, ClusteringParamSet.  
Imported table: blue oval, auto-processing table, the processing depends on the importing of external files. e.g. process of Clustering requires output files from kilosort2.  
Computed table: red circle, auto-processing table, the processing does not depend on files external to the database, commonly used for data analyses.     
Part table: plain text, as an appendix to the master table, all the part entries of a given master entry represent a intact set of the master entry. e.g. Unit of a CuratedClustering.

**Dependencies**:  

One-to-one primary: thick solid line, share the exact same primary key, meaning the child table inherits all the primary key fields from the parent table as its own primary key.     
One-to-many primary: thin solid line, inherit the primary key from the parent table, but have additional field(s) as part of the primary key as well
secondary dependency: dashed line, the child table inherits the primary key fields from parent table as its own secondary attribute.

In [ ]:
# plot diagram of tables in multiple schemas
dj.Diagram(subject) + dj.Diagram(acquisition) + dj.Diagram(ephys) + dj.Diagram(ephys_element)

In [ ]:
# plot diagram of selected tables and schemas
dj.Diagram(subject.Subject) + dj.Diagram(acquisition.Session) + dj.Diagram(ephys) + dj.Diagram(ephys_element)

In [ ]:
# plot diagram with 1 additional level of dependency downstream
dj.Diagram(subject.Subject) + 1

In [ ]:
# plot diagram with 2 additional levels of dependency upstream
dj.Diagram(ephys_element.EphysRecording) - 2

+ `describe()`: show table definition with foreign key references.

In [ ]:
ephys_element.EphysRecording.describe();

+ `heading`: show attribute definitions regardless of foreign key references

In [ ]:
ephys_element.EphysRecording.heading

# Major tables for the ephys workflow

In [ ]:
# subject, session, and ephys session
dj.Diagram(subject.Subject) + dj.Diagram(acquisition.Session) + dj.Diagram(ephys.EphysSession)

In [ ]:
subject.Subject.describe();

In [ ]:
acquisition.SessionStarted.describe();

In [ ]:
acquisition.Session.describe();

In [ ]:
# saves the data directory ephys data
ephys.EphysSession.describe();

+ [`ephys_element`](https://github.com/datajoint/element-array-ephys): Neuropixel based probe and ephys information. Check [this link](https://github.com/datajoint/element-array-ephys/tree/main/element_array_ephys) for definitions of the tables

In [ ]:
dj.Diagram(probe_element) + dj.Diagram(ephys_element)

## Summary and next step

+ This notebook introduced the overall structures of the schemas and tables in the workflow and relevant tools to explore the schema structure and table definitions.

+ In the next notebook [02-process-ephys-workflow](02-process-ephys-workflow.ipynb), we will further introduce the detailed steps running through the pipeline and table contents accordingly.